## Introduction

The goals of this lab are: 

   * Set up and analyze a firewall.
   * Understand symmetric and public-key encryption by setting up and analyzing TLS for secure TCP connections.
   * Set up and analyze VPNs for secure network tunneling.
   * Understand the principles of sender authentication and message integrity by applying and verifying digital signatures. 

The lab has several **milestones**. Make sure you reach each one before advancing to the next but try to skim through the whole lab before you start.

For delivery, submit a PDF report where you answer **only** to those steps that are marked with __REPORT__.

## System Setup  

In this lab, you will use a system setup as shown in [Figure 1](#figure_1). It is similar to lab5, but the servers now have private IP addresses. Your job is to set up a firewall to allow secure access from external networks. Then set up a website with TLS support. Finally, you will setup a VPN to establish a secure network tunnel between two private networks.


<a id='figure_1'></a>

|<img src="figures/lab6.png" width="1000" height="1000" />|
|:--:| 
| *Figure 1: Lab6 System Setup* |




* To copy files from or to containers, use `scp` or the mounted volumes. Read through the "docker-compose.yml" to find out which directories are mounted.

* The OSPF routing protocol is already configured. NAT is also configured in "router1" and "router2".

* To save your configurations, run the script: `sudo sh ~/work_dir/save.sh` inside the corresponding container.

In [ ]:
import sys
!{sys.executable} -m pip install cryptography
from test_lab6 import TestLab6
check_progress = TestLab6()

# Milestone 1 -- Firewall

In [lab5](../lab5/figures/lab5.png), for simplicity, we assigned public IP address to the webserver, which makes it available to any external network (i.e., the IP address is publicly routable across the entire internet), and anyone can access it in an unconstrained manner. But this might not be ideal.

We should place our resources in an internal network ([Figure 1](#figure_1)) and should carefully scrutinize any access from an external network through a firewall.

<a id='sec:1.1'></a>

## 1.1 [Port Forwarding](https://en.wikipedia.org/wiki/Port_forwarding)

The first step in building the firewall is to allow access to certain services residing on a private (protected) network. We can use port forwarding to remap the public IP and port to that service's private IP address and port.

* In __"pclient2"__, verify that you can __not__ access the webserver (neither through the private IP of "webserver" nor through the public IP of "router0"):

```bash
wget http://10.20.30.3 --header "Host: www.ttm4200.com"
wget http://129.168.1.2 --header "Host: www.ttm4200.com"
```

* In __"router0"__, set up port forwarding to redirect any HTTP or HTTPS requests to the webserver. That means to set up a packet filter to inspect incoming packets: if a packet has a destination port of 80 (HTTP) or 443 (HTTPS), forwarded it to the webserver of the private IP 10.20.30.3:


```bash
sudo nft add table nat
sudo nft 'add chain nat postrouting { type nat hook postrouting priority 100 ; }'
sudo nft 'add chain nat prerouting { type nat hook prerouting priority -100; }'
sudo nft 'add rule nat prerouting ip daddr 129.168.1.2 tcp dport { 80, 443 } dnat to 10.20.30.3'
sudo nft add rule nat postrouting masquerade
```

> See this [tutorial](https://jensd.be/1086/linux/forward-a-tcp-port-to-another-ip-or-port-using-nat-with-nftables) or this [tutorial](https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/7/html/security_guide/sec-configuring_port_forwarding_using_nftables) to fully understand the previous commands.

* In __"pclient2"__, verify that you can access the webserver by retrieving the content of its HTML page:

```bash
wget http://129.168.1.2 --header "Host: www.ttm4200.com"
```

In [ ]:
check_progress.test_1_1()

<a id='sec:1.2'></a>

## 1.2 Prevent Probing/Scanning a Network

One way to prevent probing/scanning your network is to block ICMP ping:


* In __router0__, drop all incoming ICMP echo requests, using nftables. 

> To solve this, you need to understand nftables. These are some suggested sources to read more ([link1](https://wiki.nftables.org/wiki-nftables/index.php/Quick_reference-nftables_in_10_minutes), [link2](https://www.linkedin.com/pulse/comprehensive-guide-nftables-leading-packet-filtering-arash-shirvar/), [Link3](https://www.youtube.com/watch?v=lHLPZlZLWgs))

* Then, make sure you can not ping "router0".


* __Optional:__ What is the downside of disabling ICMP? 
>[HINT](https://blog.paessler.com/disabling-icmp-and-snmp-wont-increase-security-but-will-impact-network-monitoring)

In [ ]:
check_progress.test_1_2()

<a id='sec:1.3'></a>

## 1.3 Filtering Inbound Traffic

A firewall on a home router allows you to initiate communications from internal devices but restricts any communication initiation from external networks. Your task in this section is to implement this firewall.


* Set up a firewall in __"router1"__ that:

    * Allow all connections initiated from internal devices.
    * Allows only established or [related](https://serverfault.com/questions/1037091/should-related-connections-always-be-allowed-in-iptables-ip6tables-nftables-et) connections from external networks
    * Otherwise, drop all incoming connections from external networks.
    >[HINT](https://wiki.nftables.org/wiki-nftables/index.php/Simple_ruleset_for_a_home_router)

In [ ]:
check_progress.test_1_3()

Q1. __REPORT:__ Briefly explain the packet filters above ([1.1](#sec:1.1), [1.2](#sec:1.2). [1.3](#sec:1.3)). Your explanation should also address:

   * Whether the packet filter is [stateful](https://en.wikipedia.org/wiki/Stateful_firewall) or stateless, and why?
   * Where the packet filter applies in the network stack, and why?
       
       > HINT: There are 5 points in the network stack (aka, [hooks](https://wiki.nftables.org/wiki-nftables/index.php/Netfilter_hooks)), where you can apply a rule of a packet filter:
       >
       > __- prerouting__: inspects all packets entering the system.
       >
       > __- input__: inspects packets delivered to the local system.
       >
       > __- forward__: inspects packets forwarded to a different host.
       >
       > __- output__: inspects packets sent by local processes.
       >
       > __- postrouting__: inspects all packets leaving the system.

# Milestone 2 -- Securing TCP Connection: TLS

In this section, you will set up a secure TCP connection using Transport Layer Security (TLS), commonly confused with its predecessor Secure Sockets Layer (SSL), which is deprecated.
Then you will analyze a trace of TLS records to investigate the various records types and the fields in these records.

You will set up a web server that supports TLS (HTTPS: HTTP over TLS).
Then investigate how cryptography enhances HTTP with security (confidentiality, data integrity, and end-point authentication).

## 2.1 Eavesdropping Insecure Connection

* The "webserver" is set up with _nginx_ but unsecured without TLS. Thus any established connection is susceptible to eavesdropping and tampering by an outside party.

To verify that: 

* Start packet capturing on __"router3"__ (as the outside party, or "Trudy") on __"ether0"__ interface. Retrieve the content of the webserver from "pclient2". Stop packet capturing and open the capture file in Wireshark.

* Find the HTTP response (HTTP/1.1 200 OK) sent from the "webserver" to "pclient2". Can you read the response in cleartext (line-based text data)? 

* Additionally, you can check that a web server is not secure through a browser, as in Figure 2.


|<img src="figures/not_secure.png" width="800" height="800" />|
|:--:| 
| *Figure 2: Insecure TCP connection* |

## 2.2 Creating TLS Certificate

The first step in setting up a website with TLS support is to get a digital certificate signed by a [certification authority (CA)](https://en.wikipedia.org/wiki/Certificate_authority).
However, we can create a Self-Signed SSL Certificate using [OpenSSL](https://www.openssl.org/).

> __NOTE:__ Using OpenSSL to generate certificates doesn't mean we are using the deprecated SSL protocol.
> In fact, we can generate _certificates_ that can be used by both SSL and TLS _protocols_.

* First, in the __"webserver"__, generate the server's private key with the following command:

```bash
sudo openssl genrsa -out /etc/ssl/private/ttm4200-selfsigned.key 2048
```

   This will generate an RSA key, 2048 bits long.
   This key is required to sign the certificate.
   Typically, a certificate is signed by a certificate authority (CA) using CA's private key, which will allow our browsers and other equipment to trust such keys (recall the importance of CAs on page 662 of the book).
   However, we will use the private key to sign the certificate in our case.
   
   > __Extra:__ If you are interested in freely generating a worldwide valid key, you may try [Let's Encrypt](https://letsencrypt.org). However, take care that some of the steps may differ.



* Now, you will generate the TLS certificate.
   To do it, run the following command and provide the public information that will be embedded in the certificate.
   Feel free to change the information except for the Common Name, which must correspond to the server's domain name.

```bash
sudo openssl req -x509 -sha256 -nodes -days 60 -key /etc/ssl/private/ttm4200-selfsigned.key -out /etc/ssl/certs/ttm4200-selfsigned.crt

Country Name (2 letter code) [AU]:NO
State or Province Name (full name) [Some-State]:Trøndelag
Locality Name (eg, city) []:Trondheim
Organization Name (eg, company) [Internet Widgits Pty Ltd]:NTNU
Organizational Unit Name (eg, section) []:IIK
Common Name (e.g. server FQDN or YOUR name) []:www.ttm4200.com
Email Address []:murad@ttm4200.com
```

   > More information about the command:
   >
   > __[-x509](https://en.wikipedia.org/wiki/X.509)__: define the format of public-key certificates. (<https://en.wikipedia.org/wiki/X.509>)
   >
   > __-sha256__: use a stronger hash algorithm
   >
   > __-nodes__: skip securing the certificate with a passphrase.
   >
   > __-days__ 60: specify the validity of the certificate
   >
   > __-key__: specify the private key to sign this certificate
   >
   > __-out__: specify the path and name of the certificate.
   >
   > __-Emai Address__: Use an email address with your team number (e.g., alex@team10.com). We will check this in the capture file to prove that your team generated the certificate.

* Create a Diffie–Hellman (D-H) Parameter:
   
```bash
sudo openssl dhparam -out /etc/ssl/certs/dhparam.pem 2048
```

   > "Using Diffie–Hellman key exchange will generate a random and unique session key for encryption and decryption that has the additional property of forward secrecy: if the server's private key is disclosed in future, it cannot be used to decrypt the current session, even if the session is intercepted and recorded by a third party." [Reference](https://en.wikipedia.org/wiki/Transport_Layer_Security).


## 2.3 Configuring the Web Server to Support TLS

The _nginx_ service installed in the "webserver" is configured at "/etc/nginx/site-available/ttm4200" to support HTTP.
You need to adjust the configuration to use TLS and redirect HTTP requests to HTTPS, thus enforcing the use of the TLS certificate. 

* Start by filling the following template. Keep in mind that HTTPS uses a different port number than HTTP.

    >[HINT](http://nginx.org/en/docs/http/configuring_https_servers.html)

```bash
server {
     #enable ssl on listening socket (IPv4, default_server)
     listen ====fill in here====;

     #enable ssl on listening socket (IPv6, default_server)
     listen ====fill in here====;

     #location of the server certificate
     ssl_certificate ====fill in here====;

     #location of the private key
     ssl_certificate_key ====fill in here====;

     #location of the dh parameter
     ssl_dhparam ====fill in here====;

     root /var/www/ttm4200;
     index ttm4200_index.html;
     server_name www.ttm4200.com;
     location / {
        try_files $uri $uri/ =404;
        } 
     error_page 500 502 503 504 /50x.html;
     location = /50x.html {
           root /var/www/ttm4200;
        index 50x.html;
        }
  }

   # catch-all http requests and redirect them to https
   server{
      listen 80 default_server;
      listen [::]:80 default_server;
      server_name www.ttm4200.com;
      return 301 https://www.ttm4200.com$request_uri;
      }

   ```


* Check that the configuration's syntax is correct (`sudo nginx -t`) and if so, restart _nginx_.

* In your VM, access the website through a browser. Because we are using a self-signed certificate (i.e., not signed by any trusted CA), the browser will be unable to verify the certificate presented by the webserver. It will issue a warning, as in Figure 3.




|<img src="figures/ssl_warning.png" width="600" height="600" />|
|:--:| 
| *Figure 3: Self-signed certificate warning* |


* Click on _Advanced_ $\rightarrow$ _View Certificate_ to display certificate information. 

> __Note:__ If you already pressed _Accept the Risk and Continue_ you can still view the certificate by clicking on the lock on the address bar. 
  
* __Self-Check:__ Why are the "Subject Name" and "Issuer Name" the same? 

* __Self-Check:__ What algorithm is used for public-key encryption? 

* __Self-Check:__ What is the signature algorithm? 

* __Self-Check:__ What hash function is used to produce the certificate fingerprints? 


In [ ]:
check_progress.test_2_3()

## 2.4 Analyzing TLS protocol.


To analyze TLS, you need to capture packets in a TLS session. 

* Start packet capturing in __"router3"__ on "ether0" interface, and dump the capture to a file named "https_tls.pcap".



```bash
sudo tcpdump -i ether0 -w https_tls.pcap
```

* In __"pclient2"__, retrieve the content from the "webserver" using HTTPS and skip the certificate verification:

```bash
wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate
```



* Stop packet capturing in "router3". Copy the capture file to the "lab6" directory, then open it in Wireshark. Apply the display filter `tls` to show only the frames that have TLS records. Then answer the following questions:

   >_NOTE: Keep in mind that a single packet may contain one or more TLS records._

* Locate the first message in the TLS handshake (_ClientHello_ record), which the client sends to initiate a session with the server. Then answer the following:
    
    * What is the packet number of _ClientHello_ record?
    * What is the value of the content type?
    * Locate the nonce (also known as a "Random"), consisting of a random number and a Unix timestamp. What is the value of the nonce? How long (in bytes) is the nonce?
    * Locate the [cipher suite](https://en.wikipedia.org/wiki/Cipher_suite) list, which contains the combinations of cryptographic algorithms supported by the client in order of the client's preference (favorite choice first). How many cipher suites are advertised in the Client Hello record? 
    
    * __Self Check:__ Locate the "supported_versions" Extension. Why is the client requesting a TLS 1.2 handshake when it can support TLS 1.3? [HINT](https://www.thesecmaster.com/decoding-tls-1-3-protocol-handshake-with-wireshark/)

In [ ]:
# What is the packet number of _ClientHello_ record?
ClientHello_packet_number =  # your answer (as integer)

# What is the value of the content type?
ClientHello_content_type =  # your answer (as integer)

# Locate the nonce (also known as a "Random"), consisting of a random number and a Unix timestamp. What is the value of the nonce? 
ClientHello_nonce_value =  # your answer (as a string of hexadecimal value)
# How long (in bytes) is the nonce? 
ClientHello_nonce_length =  # your answer (as integer) 

# How many cipher suites are advertised in the Client Hello record? 
Number_of_cipher_suites =  # your answer (as integer) 


check_progress.test_2_4_1(ClientHello_packet_number, ClientHello_content_type, ClientHello_nonce_value , ClientHello_nonce_length, Number_of_cipher_suites)

* Locate the _ServerHello_ record and expand it. Then answer the following:

    * What is the packet number containing the _ServerHello_ record?
    * Which TLS version does the server support?

    * Find the chosen cipher suite (the server-chosen cipher suite from the client's advertised list) and specify the selected algorithms:
        * The symmetric-key algorithm (bulk encryption algorithm) is used to encrypt the sent data.
        * The hash algorithm (message authentication code algorithm)
        
        
        
* __Self Check:__ Can you locate the _Certificate_, _Server Key Exchange_, and _Client Key Exchange_ records? Why?

In [ ]:
# What is the packet number containing the _ServerHello_ record?
ServerHello_packet_number =   # your answer (as integer)

# Which TLS version does the server support?
Server_suppored_TLS_version =   # your answer (either 1.0, 1.1, 1.2, 1.3)

# Find the chosen cipher suite in the ServerHello_and specify the selected algorithms:
symmetric_key_algorithm =  # your answer (as string)
hash_algorithm =  # your answer (as string)


check_progress.test_2_4_2(ServerHello_packet_number, Server_suppored_TLS_version, symmetric_key_algorithm, hash_algorithm)

Q2. __REPORT:__ Draw a sequence diagram of all TLS __records__ exchanged during TLS handshake. Then:
    
   * Explain the functionality of each TLS record.
   * Explain how the application data is encrypted.
 
>__NOTE:__ Keep in mind that a single packet may contain one or more records. The question is about TLS __records__, not packets or frames.

# Milestone 3 -- Virtual Private Network (VPN)


To establish a protected network over a public internet connection, we can set up a VPN to encrypt the traffic and allow access to services on private networks. For example, the "Openstack" VMs are only accessible via NTNU private network, and you had to set up a VPN connection to access them.

This lab will use WireGuard, free and open-source VPN software, which it is faster, simpler, and more efficient than IPsec and OpenVPN. It is already installed in all containers of this lab. You __must__ read this [page](https://www.wireguard.com/) (<https://www.wireguard.com/>) to get a conceptual overview of WireGuard.


* First, remove port forwarding from __"router0"__ to prevent external access to internal resources in "ttm4200_private_net":

```bash
sudo nft flush ruleset
```

* In __"pclient2"__, verify that you can __not__ access the webserver:

```bash
wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate
wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate
```


In [ ]:
check_progress.test_3_0()

## 3.1 VPN Setup: WireGuard

In the following steps, you will establish a VPN connection from __"pclient2"__ (WireGuard client) to __"router0"__ (WireGuard server). Refer back to [Figure 1](#figure_1):


* Generate public and private keys for both the sever (__"router0"__) and the client (__"pclient2"__) (i.e., these commands need to be executed in both containers):

```bash
umask 077
wg genkey | tee privatekey | wg pubkey > publickey
```

   > More information about the command (<https://man7.org/linux/man-pages/man8/wg.8.html>):
   >
   > __- [umask 077](https://en.wikipedia.org/wiki/Umask)__: Set file permission for newly created files to only the user has "read," "write," and "execute" permissions (completely private).
   >
   > __- wg genkey__: Generates a random private key in base64.
   >
   > __- | tee privatekey__: Read the output of the generated private key and write it to a file named "privatekey" .
   >
   >__- | wg pubkey > publickey__: Calculates a public key from the corresponding private key and write it to a file named "publickey"

* In the server (__"router0"__), create a configuration file "/etc/wireguard/wg0.conf", and add the following lines::

```bash
[Interface]
PrivateKey = <Private key of "router0">
#We will use a VPN subnet of 10.20.30.128/25, use 10.20.30.130 for the sever
#and 10.20.30.131 for the client.
Address = 10.20.30.130/25
#Standard port for WireGuard (optional)
ListenPort = 51820
#Accept every packet on the tunnel interface, and masquerade outgoing 
#packets with a public IP
PostUp = iptables -A FORWARD -i wg0 -j ACCEPT 
PostUP = iptables -t nat -A POSTROUTING -o ether1 -j MASQUERADE
#Delete the NAT rule when removing the VPN
PostDown = iptables -D FORWARD -i wg0 -j ACCEPT
PostDown = iptables -t nat -D POSTROUTING -o ether1 -j MASQUERADE

[Peer]
PublicKey = <Public key of "pclient2">
AllowedIPs = 10.20.30.131
#Send a keepalive packet every 25 seconds
PersistentKeepalive = 25
```
   >_HINT: you can run `cat privatekey`, then copy the key and paste it into the config file. The same goes for the publickey._


* In the client (__"plient2"__), create a configuration file "/etc/wireguard/wg0.conf", and add the following lines:

```bash
[Interface]
PrivateKey = <Private key of "pclient2">
Address = 10.20.30.131/25

[Peer]
PublicKey = <Public key of "router0">
#Public ip address of the server "router0" and ListenPort
Endpoint = 129.168.1.2:51820
# Route all traffic with destination IP address in the subnet
# of (10.20.30.0/24) through this Wireguard tunnel
AllowedIPs = 10.20.30.0/24
```



* Bring up the WireGuard tunnel interface, in __both__ the server and the client:

```bash
sudo wg-quick up /etc/wireguard/wg0.conf
```

* Verify that there is a new interface "wg0" added:

```bash
ip address
```

* Verify that your VPN is set up correctly by accessing the webserver from __"pclient2"__:

```bash
ping 10.20.30.3

wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate
```

In [ ]:
check_progress.test_3_1()

## 3.2 Analyzing VPN

* Start packet capturing on __"router3"__ (as the outside party, or "Trudy") on __"ether0"__ interface (refer back to [Figure 1](#figure_1)), and dump the capture to a file named "wireguard_https.pcap":



* Bring down WireGuard tunnel interface, then bring it up to capture the handshake (in __both__ the server and the client):

```bash
sudo wg-quick down /etc/wireguard/wg0.conf
sudo wg-quick up /etc/wireguard/wg0.conf
```

* In __"pclient2"__, retrieve the content from the "webserver, and then stop packet capturing. 

```bash
wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate
```

Copy the capture file to the "lab6" directory, then open it in Wireshark. Apply the display filter `wg` to show only WireGuard packets. Then answer the following questions:


* Locate the "Handshake Initiation" packet, which corresponds to the first message from Initiator ("pclient2" in our case) to Responder ("router0" in our case). Then answer the following:

    * What is the packet number of the "Handshake Initiation" packet?
    * What is the used transport protocol? 
    * What is the destination port? 
    
    * __Self Check:__ Does the destination port corresponds to the "ListenPort" in the WireGuard server?
    
    * How many WireGuard fields are in the "Handshake Initiation" packet?
    
    * __Self Check:__ What are these fields? [HINT](https://www.wireguard.com/protocol/#first-message-initiator-to-responder)

* Locate the "Handshake Response" packet, which corresponds to the second message from Responder to Initiator. 

    * What is the packet number of the "Handshake Response" packet?
    * How many WireGuard fields are in the "Handshake Response" packet?
    
    * __Self Check:__ What are these fields? 


* Locate a "Transport Data" packet, corresponding to the encapsulated data.

    * What is the packet number of the "Transport Data" packet?
    * How many WireGuard fields are in the "Transport Data" packet?
    
    * __Self Check:__ What are these fields? 

* Locate a "Keepalive" message.   

    * What is the packet number of a "Keepalive" packet?
    * How many WireGuard fields are in the "Keepalive" packet?
    
    * __Self Check:__ What are these fields? 

In [ ]:
################  Handshake Initiation ########################################
# What the packet number of the "Handshake Initiation" packet?
Handshake_Initiation_packet_number =   # your answer (as integer)

#  What is the used transport protocol? 
wireguard_transport_protocol=  # your answer (as string)

# What is the destination port? 
Handshake_Initiation_destination_port=  # your answer (as integer)

# How many WireGuard fields are in the "Handshake Initiation" packet?
Handshake_Initiation_num_fields =  # your answer (as integer)


################  Handshake Response ########################################
# What is the packet number of the "Handshake Response" packet?
Handshake_Response_packet_number =  # your answer (as integer)

# How many WireGuard fields are in the "Handshake Response" packet?
Handshake_Response_num_fields =   # your answer (as integer)


################  Transport Data ########################################
# What is the packet number of a "Transport Data" packet?
Transport_Data_packet_number =   # your answer (as integer)

# How many WireGuard fields are in the "Transport Data" packet?
Transport_Data_num_fields =   # your answer (as integer)


################  Keepalive ########################################
# What is the packet number of a "Keepalive" packet?
Keepalive_packet_number =   # your answer (as integer)

# How many WireGuard fields are in the "Keepalive" packet?
Keepalive_num_fields =  # your answer (as integer)

check_progress.test_3_2(Handshake_Initiation_packet_number, 
                        wireguard_transport_protocol, 
                        Handshake_Initiation_destination_port,
                        Handshake_Initiation_num_fields,
                       Handshake_Response_packet_number,
                       Handshake_Response_num_fields,
                       Transport_Data_packet_number,
                       Transport_Data_num_fields,
                       Keepalive_packet_number,
                       Keepalive_num_fields)

Q3. __REPORT:__ Which of the following security requirements does WireGuard provide/satisfy in this lab? Briefly explain why?

   * Confidentiality?
   * Integrity?
   * Authentication?
   * Forward Secrecy?
   * Traffic Type Obfuscation?

Q4. __REPORT:__ What do you think would happen if you use the public IP address (i.e., `wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate`) while using VPN? What would happen if you also perform port forwarding on "router0" (as you did in [section 1.1](#sec:1.1))? Briefly explain why?
>[HINT](https://stackoverflow.com/questions/65444747/what-is-the-difference-between-endpoint-and-allowedips-fields-in-wireguard-confi)


__Optional__ If you are interested, see how some of your colleagues used WireGuard in their master thesis [Link](https://ntnuopen.ntnu.no/ntnu-xmlui/bitstream/handle/11250/2834784/no.ntnu%3ainspera%3a80310926%3a20549418.pdf?sequence=1&isAllowed=y), [Link](https://ieeexplore.ieee.org/document/9289900).

__Extra Credit:__ WireGuard supports optional Pre-shared Symmetric key Mode, which provides sufficient protection for post-quantum cryptography. According to the WireGuard protocol description: "If an additional layer of symmetric-key crypto is required (for, say, post-quantum resistance), WireGuard also supports an optional pre-shared key that is mixed into the public key cryptography. When the pre-shared key mode is not in use, the pre-shared key value is assumed to be an all-zero string of 32 bytes." (ref:<https://www.wireguard.com/protocol/>). Your __task__ is to reconfigure WireGuard with a pre-shared key (use the same setup as before, but include a pre-shared key). Submit a screenshot of your configuration, a screenshot of the output of `sudo wg show`. Briefly explain how using a Pre-shared key augments WireGuard security in case of quantum computing advances.

__Optional__ [Interesting article](https://www.technologyreview.com/2021/11/03/1039171/hackers-quantum-computers-us-homeland-security-cryptography/).


# Milestone 4 -- Digital Signature


In this milestone, you will apply a digital signature to provide document authentication (verifying that a known sender created the document) and integrity (verifying that the document was unaltered in transit).
We will use [Pretty Good Privacy (PGP)](https://en.wikipedia.org/wiki/Pretty_Good_Privacy) protocol for creating and verifying signatures.

One common and free implementation of PGP is [GNU Privacy Guard (GPG)](https://en.wikipedia.org/wiki/GNU_Privacy_Guard), also known as [GnuPG](https://gnupg.org/).
We will use the command-line tool, which can be installed in your VM (no need for `docker` in this milestone):

```bash
sudo apt-get install gnupg
```

## 4.1 Public/Private Keys

* Create a PGP public/private key pair with the following command:

```bash
gpg --gen-key
```
   
   Then enter a name (e.g., your team's name), email address (e.g., one of your emails), and a passphrase to protect your private key (remember this passphrase!).
   By default, it will create an RSA key with 3072 bits.
   You will use the private key to sign a document, while others can use your public key to verify your signature.

* Export your public key as an ASCII file so you can share it with others:
   
```bash
gpg --export  youremail > teamName_public.key
```
   
* Import and trust others' public keys.
   A public key belongs to this course at lab6 directory ("lab6/pgp/ttm4200_pubkey.asc"). 
   You should import it, and trust it:

```bash
gpg --import ttm4200_public.key
gpg --edit-key ttm4200
trust
5
y
quit
```

## 4.2 Verifying Digital Signature

Q5. __REPORT:__ There is a pdf document at "lab6/pgp/verify_me.pdf" that was digitally signed with the private key of ttm4200, where the signature file is at "lab6/pgp/verify_me.pdf.sig". Verify the document's signature, then explain how this verification proves (or does not prove) the document's __integrity__ and __authenticity__.

>__Note:__ assume the public-key hasn't been tampered with. 

>__[HINT](https://www.gnupg.org/gph/en/manual/x135.html)__


## 4.3 Creating Digital Signature & Encryption

Q6. __REPORT:__ Explain how you can encrypt your report such that only the owner of the private key of ttm4200 can read the encryption (i.e., [how PGP encryption works?](https://en.wikipedia.org/wiki/Pretty_Good_Privacy#/media/File:PGP_diagram.svg)). Then (when you finish writing the delivery report), digitally sign your report (__detached signature__), and encrypt your report such that only the owner of the private key of ttm4200 can read the encryption. Submit the following files:

* Your report (unencrypted version).
* Your public key.
* The signature file.
* The encrypted report.